In [2]:
from pathlib import Path


# list of all -merged folders in stubs
merged_folders = [x for x in Path("stubs").iterdir() if x.is_dir() and "-merged" in x.name]

In [3]:
NO_DOCS = ["(self, *args, **kwargs)", "(cls, *args, **kwargs)", "(*args, **kwargs)"]


def scan_py_file(py):
    with py.open(encoding="utf-8") as f:
        for line in f:
            if any(x in line for x in NO_DOCS):
                yield py, line


def scan_folder(folder):
    for py in folder.glob("*.py"):
        yield scan_py_file(py)

In [5]:
for folder in merged_folders:
    # if   latest or  1.20.0 in folder name
    if "latest" in folder.name or "1_20_0" in folder.name:
        print(folder.name)

    continue

micropython-latest-esp32-merged
micropython-latest-esp8266-merged
micropython-latest-rp2-merged
micropython-latest-stm32-merged
micropython-v1_20_0-esp32-merged
micropython-v1_20_0-esp32-OTA-merged
micropython-v1_20_0-esp32-S3-merged
micropython-v1_20_0-rp2-merged
micropython-v1_20_0-rp2-PICO-merged
micropython-v1_20_0-rp2-PICO_W-merged
micropython-v1_20_0-rp2-PIMORONI_PICOLIPO_16MB-merged
micropython-v1_20_0-samd-ADAFRUIT_FEATHER_M4_EXPRESS-merged
micropython-v1_20_0-samd-ADAFRUIT_ITSYBITSY_M4_EXPRESS-merged
micropython-v1_20_0-samd-MINISAM_M4-merged
micropython-v1_20_0-samd-SEEED_WIO_TERMINAL-merged
micropython-v1_20_0-stm32-merged
micropython-v1_20_0-stm32-PYBV11-merged


In [7]:
not_found = {}

for folder in merged_folders:
    if "latest" not in folder.name and "1_20_0" not in folder.name:
        continue

    for result in scan_folder(folder):
        for file, line in result:
            # append line to dict of lines
            line = line.rstrip()
            not_found.setdefault(line, []).append(file)

In [8]:
# write the output to a file

with open("not_found.txt", "w") as fp:
    for k in not_found.keys():
        fp.write(f"----------------------------------------------------------\n")
        fp.write(f"{k}\n")
        for f in not_found[k]:
            fp.write(f"  -  {f}\n")
        fp.write("\n")

In [12]:
# for results in scan_folder(Path("stubs\micropython-v1_20_0-stm32-merged")):
#     for py, line in results:
#         print(py, line, end="")

stubs\micropython-v1_20_0-stm32-merged\binascii.py def crc32(*args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def popitem(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def pop(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def values(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def setdefault(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def update(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def copy(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def clear(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def keys(self, *args, **kwargs) -> Any:
stubs\micropython-v1_20_0-stm32-merged\collections.py     def get(self, *args, **kwargs) -> Any:
stubs\micropython-v1